In [1]:
import sys, os, pickle
import time
import copy
import pandas as pd
import numpy as np
from multiprocessing import Pool, cpu_count


from lmp_apps.pbbm.lung_pbbm import PBPK_Model_Virtual_Trial
from lmp_apps.pbbm.main import run_pbbm_simulations, process_simulation_solutions

from lmp_apps.lung_deposition.main import run_deposition_simulations
from lmp_apps.population.attributes import SubjectPhysiology
from lmp_apps.population.variability import Variability
from lmp_apps.api.api import API
from lmp_apps.study.study import Product, Study
from lmp_apps.vbe.bioequivalence import BioequivalenceAssessor
from lmp_apps.efficacy.fev1model import COPD_FEV1
from lmp_apps.visualisation.plots import TrialVisualizer
from lmp_apps.utils.config import simulation_settings

import warnings
warnings.filterwarnings('ignore')



In [ ]:
SAVE_FLAG = False

output_dir = '/projects/op/se_op/scaleupsgrm/LungDeposition/LMP_Simulations_Results/'


NUM_CORES = 1
N_SUBJECTS = 1
N_TRIALS_BE = 50
distributed = False


study = Study(study_type='CHARCOAL', trial_size = N_SUBJECTS, n_trials = N_TRIALS_BE)
settings = simulation_settings(ENABLE_VARIABILITY = True, simtime = 12.0, study_type = study.study_type)
# --- Setup APIs and Products ---

bd_base, gp_base, ff_base = API(name="BD"), API(name="GP"), API(name="FF")
base_apis={"BD": bd_base, "GP": gp_base, "FF": ff_base}

# Create deep copies of the API objects for each product to ensure independence
test_apis = copy.deepcopy(base_apis)
ref_apis = copy.deepcopy(base_apis)

test_product = Product("Test", base_apis=[
    {'api': test_apis["BD"], 'dose_pg': 330.34e6, 'mmad': 4.06, 'gsd': 1.67, 'propellant': 'PT210', 'device': 'DFP', 'usp_depo_fraction': 41.08},
    #{'api': test_apis["GP"], 'dose_pg': 14.42e6, 'mmad': 3.94, 'gsd': 1.67, 'propellant': 'PT210', 'device': 'DFP', 'usp_depo_fraction': 38.06},
    #{'api': test_apis["FF"], 'dose_pg': 9.68e6, 'mmad': 4.0, 'gsd': 1.69, 'propellant': 'PT210', 'device': 'DFP', 'usp_depo_fraction': 39.50}
], study = study)
ref_product = Product("Reference", base_apis=[
    {'api': ref_apis["BD"], 'dose_pg': 329.04e6, 'mmad': 3.53, 'gsd': 1.61, 'propellant': 'PT010', 'device': 'DFP', 'usp_depo_fraction': 41.26},
    #{'api': ref_apis["GP"], 'dose_pg': 14.46e6, 'mmad': 3.4, 'gsd': 1.62, 'propellant': 'PT010', 'device': 'DFP', 'usp_depo_fraction': 38.18},
    #{'api': ref_apis["FF"], 'dose_pg': 9.62e6, 'mmad': 3.4, 'gsd': 1.66, 'propellant': 'PT010', 'device': 'DFP', 'usp_depo_fraction': 38.28}
], study = study)

#products = [ref_product, test_product]
products = [ref_product]
print("--- Starting Virtual BE Trial ---")
print(f"Variability Enabled: {settings.ENABLE_VARIABILITY}")

# 1. Generate the virtual population
variability = Variability(enable_variability=settings.ENABLE_VARIABILITY)
subjects = [SubjectPhysiology(i, variability) for i in range(1, study.trial_size + 1)]

# 2. Pre-compute lung depositions in parallel
deposition_pool = run_deposition_simulations(subjects, products, NUM_CORES, distributed )

# 5. Assemble and Process Results
simulation_outputs = run_pbbm_simulations(subjects, products, deposition_pool, variability, settings, NUM_CORES, distributed = False)
#simulation_outputs = run_pbbm_simulations_ORG(subjects, products, deposition_pool, variability, settings, NUM_CORES)

final_pk_params = [item for output in simulation_outputs for item in output['pk_params']]

results_df = pd.DataFrame(final_pk_params)

processed_profiles_df = process_simulation_solutions(simulation_outputs)

if SAVE_FLAG:
    results_df.to_csv(os.path.join(output_dir, simname + '_End_Results.csv'))

    processed_profiles_df.to_csv(os.path.join(output_dir,  simname + '_Time_Results.csv'))

NameError: name 'Study' is not defined

In [3]:
results_df

,Region,Compartment,Regional_AUC,Systemic_AUC,Systemic_Cmax,Dose,ET,C/P,Regional Deposition,Subject,Product,API
0,ET,Epithelium,0.000000e+00,950.158701,464.851684,3.284875e+08,0.568579,7.016573,1.867711e+08,1,Reference,BD
1,ET,Tissue,0.000000e+00,950.158701,464.851684,3.284875e+08,0.568579,7.016573,1.867711e+08,1,Reference,BD
2,ET,Epithelium_Tissue,0.000000e+00,950.158701,464.851684,3.284875e+08,0.568579,7.016573,1.867711e+08,1,Reference,BD
3,BB,Epithelium,3.060075e+07,950.158701,464.851684,3.284875e+08,0.568579,7.016573,1.569214e+07,1,Reference,BD
4,BB,Tissue,1.883147e+06,950.158701,464.851684,3.284875e+08,0.568579,7.016573,1.569214e+07,1,Reference,BD
5,BB,Epithelium_Tissue,3.959361e+06,950.158701,464.851684,3.284875e+08,0.568579,7.016573,1.569214e+07,1,Reference,BD
6,bb_b,Epithelium,7.680852e+06,950.158701,464.851684,3.284875e+08,0.568579,7.016573,1.591917e+07,1,Reference,BD
7,bb_b,Tissue,4.327874e+06,950.158701,464.851684,3.284875e+08,0.568579,7.016573,1.591917e+07,1,Reference,BD
8,bb_b,Epithelium_Tissue,5.478838e+06,950.158701,464.851684,3.284875e+08,0.568579,7.016573,1.591917e+07,1,Reference,BD
9,Al,Epithelium,9.693237e+05,950.158701,464.851684,3.284875e+08,0.568579,7.016573,1.101051e+08,1,Reference,BD


In [ ]:
I think there are critical errors in how we construct subjects
We made two SubjectPhysiology() instances: the first one in deposition callbacks via Execution.simulate_subjects and the second one in run_pbpk callbacks.
Even though they have the same id, the variability envolved will make the subject parameters different in these two instances, which results in inaccurate results since the deposition is used in pbpk


# In deposition callbacks
def run_deposition_cb(n, setup, api_grid, invitro, subject_store):
    if not n:                       # ← guard prevents tab remounts from running it
        raise PreventUpdate
    try:
        setup = setup or {}
        study, products = build_study_and_products(setup, api_grid or {}, invitro or {})
        # settings / variability for Execution
        settings = simulation_settings(ENABLE_VARIABILITY=bool(setup.get("enable_variability", False)), simtime=24.0)
        subj0 = build_subject_from_store(subject_store or {}, enable_variability=settings.ENABLE_VARIABILITY)
        variability = subj0.variability

        results_dir = pathlib.Path(setup.get("results_dir","./results")).expanduser()
        results_dir.mkdir(parents=True, exist_ok=True)

        # parallel semantics like your codebase
        exec_mode  = setup.get("exec_mode","serial")
        use_pool   = (exec_mode == "pool")
        pool_cores = max(1, int(setup.get("n_processors",1)))
        distributed = not use_pool  # your library uses distributed=True for sequential

        if Execution is None:
            return "Execution module not found.", go.Figure(), dash.no_update

        ex = Execution(settings=settings, products=products, variability=variability)
        # Run DEPOSITION ONLY to per-subject pickles
        # This will make the first instance of list of subjects instances
        # We need to pass the population though, to ensure it runs for other cases
        ex.simulate_subjects(
            subject_ids=_subject_ids(setup),
            output_dir=str(results_dir),
            step_mode="deposition",
            distributed=distributed,
            use_pool=use_pool,
            pool_cores=pool_cores,
        )
        """ simulate_subjects signature
            def simulate_subjects(self, subject_ids, output_dir, step_mode='full', distributed=True, use_pool=False,
                                pool_cores=8, input_deposition_dir=None):
                subjects = [SubjectPhysiology(sid, self.variability) for sid in subject_ids]
                os.makedirs(output_dir, exist_ok=True)
        """
# In PBPK callbacks

def run_pbpk_cb(n, setup, api_grid, invitro, subject_store, depo_dir_from_run):
    if not n:
        raise PreventUpdate
    if run_pbbm_simulations is None or process_simulation_solutions is None:
        return "PBPK module not available yet.", go.Figure()

    try:
        setup = setup or {}
        # Build Study/Products
        study, products = build_study_and_products(setup, api_grid or {}, invitro or {})

        # Settings & variability (from Subject tab)
        settings = simulation_settings(
            ENABLE_VARIABILITY=bool(setup.get("enable_variability", False)),
            simtime=24.0
        )
        subj0 = build_subject_from_store(subject_store or {}, enable_variability=settings.ENABLE_VARIABILITY)   # 
        variability = subj0.variability

        # Exec config
        results_dir = pathlib.Path(setup.get("results_dir","./results")).expanduser()
        results_dir.mkdir(parents=True, exist_ok=True)
        exec_mode   = setup.get("exec_mode","serial")      # 'serial' | 'pool'
        use_pool    = (exec_mode == "pool")
        pool_cores  = max(1, int(setup.get("n_processors",1)))
        distributed = not use_pool

        # Subjects
        sids     = _subject_ids(setup)
        # This will make the second list of subject instances, with the same id, but values will be resampled
        subjects = [SubjectPhysiology(sid, variability) for sid in sids]
         # Where to read deposition from
        depo_base = pathlib.Path(setup.get("input_deposition_dir") or depo_dir_from_run or results_dir).expanduser()

        # Load existing deposition pickles; collect missing and compute them in one batch
        dep_map_all = {}
        missing_subjects = []
        for sid, subject in zip(sids, subjects):
            f = depo_base / f"subject_{sid}_deposition.pkl"
            if f.exists():
                with open(f, "rb") as fh:
                    d = pickle.load(fh)
                dep_entry = d["deposition_pool"][sid]
                dep_map_all[sid] = _sanitize_structure(_canonicalize_depo_for_subject(dep_entry, products), eps=0.0)
            else:
                missing_subjects.append(subject)

        if missing_subjects:
            dep_batch = run_deposition_simulations(
                missing_subjects, products,
                num_cores=(pool_cores if use_pool else 1),
                distributed=distributed
            )
            for subj in missing_subjects:
                dep_entry = dep_batch[subj.id]
                dep_map_all[subj.id] = _sanitize_structure(_canonicalize_depo_for_subject(dep_entry, products), eps=0.0)

        # Final deposition pool for PBPK: {sid: {product_name: ...}}
        deposition_pool = {sid: dep_map_all[sid] for sid in sids}


In [ ]:
# The following script is structured to split task per subject or group of subjects. and those tasks will be run with slurm job array.
could you please adapt the get_model_configuration() so that it works well for the case where we run with 1 or a small number of subjects but with a large number of products

import sys, os, pickle
import time
import copy
import pandas as pd
import numpy as np
from multiprocessing import Pool, cpu_count
import warnings
warnings.filterwarnings('ignore')

from lmp_apps.pbbm.lung_pbbm import PBPK_Model_Virtual_Trial
from lmp_apps.pbbm.main import run_pbbm_simulations, process_simulation_solutions
from lmp_apps.lung_deposition.main import run_deposition_simulations
from lmp_apps.population.attributes import SubjectPhysiology
from lmp_apps.population.variability import Variability
from lmp_apps.api.api import API
from lmp_apps.study.study import Product, Study
from lmp_apps.vbe.bioequivalence import BioequivalenceAssessor
from lmp_apps.efficacy.fev1model import COPD_FEV1
from lmp_apps.visualisation.plots import TrialVisualizer
from lmp_apps.utils.config import simulation_settings
from lmp_apps.execution.execution import Execution  



def get_model_configuration(N_SUBJECTS=10, N_TRIALS_BE=50):
    settings = simulation_settings(ENABLE_VARIABILITY=True)
    study = Study(study_type='NON_CHARCOAL', trial_size=N_SUBJECTS, n_trials=N_TRIALS_BE)
    bd_base, gp_base, ff_base = API(name="BD"), API(name="GP"), API(name="FF")
    base_apis = {"BD": bd_base, "GP": gp_base, "FF": ff_base}
    test_apis = copy.deepcopy(base_apis)
    ref_apis = copy.deepcopy(base_apis)
    test_product = Product("Test", base_apis=[
        {'api': test_apis["BD"], 'dose_pg': 330.34e6, 'mmad': 4.06, 'gsd': 1.67, 'propellant': 'PT210', 'device': 'DFP', 'usp_depo_fraction': 41.08},
        {'api': test_apis["GP"], 'dose_pg': 14.42e6, 'mmad': 3.94, 'gsd': 1.67, 'propellant': 'PT210', 'device': 'DFP', 'usp_depo_fraction': 38.06},
        {'api': test_apis["FF"], 'dose_pg': 9.68e6, 'mmad': 4.0, 'gsd': 1.69, 'propellant': 'PT210', 'device': 'DFP', 'usp_depo_fraction': 39.50}
    ], study=study)
    ref_product = Product("Reference", base_apis=[
        {'api': ref_apis["BD"], 'dose_pg': 329.04e6, 'mmad': 3.53, 'gsd': 1.61, 'propellant': 'PT010', 'device': 'DFP', 'usp_depo_fraction': 41.26},
        {'api': ref_apis["GP"], 'dose_pg': 14.46e6, 'mmad': 3.4, 'gsd': 1.62, 'propellant': 'PT010', 'device': 'DFP', 'usp_depo_fraction': 38.18},
        {'api': ref_apis["FF"], 'dose_pg': 9.62e6, 'mmad': 3.4, 'gsd': 1.66, 'propellant': 'PT010', 'device': 'DFP', 'usp_depo_fraction': 38.28}
    ], study=study)
    products = [ref_product, test_product]
    variability = Variability(enable_variability=settings.ENABLE_VARIABILITY)
    
    return study, settings, products, variability


if __name__ == "__main__":
    mode = os.environ.get("EXEC_MODE", "slurm")      # slurm, dask, pool
    pipeline_mode = os.environ.get("PIPELINE_MODE", "full")    # full, deposition, pbpk
    input_deposition_dir = os.environ.get("INPUT_DEPO", None)  # required if pbpk-only

    #output_dir = '/projects/op/se_op/scaleupsgrm/LungDeposition/LMP_Simulations_Results/'
    output_dir = sys.argv[3]
    N_SUBJECTS = 10
    N_TRIALS_BE = 50
    study, settings, products, variability = get_model_configuration(N_SUBJECTS, N_TRIALS_BE)

    executor = Execution(settings, products, variability)

    if mode == "slurm":
        start_idx = int(sys.argv[1])
        end_idx = int(sys.argv[2])
        executor.run_slurm_array(start_idx, end_idx, output_dir, pipeline_mode, input_deposition_dir)

    elif mode == "pool":
        executor.run_local_pool(study.trial_size, output_dir, pool_cores=8, step_mode=pipeline_mode, input_deposition_dir=input_deposition_dir)

    elif mode == "dask":
        executor.run_dask_parallel(study.trial_size, output_dir, pipeline_mode, n_jobs=25, input_deposition_dir=input_deposition_dir)

    else:
        print("Unknown EXEC_MODE: choose 'pool', 'dask', or 'slurm'")

#""""
#Full pipeline, all subjects, Dask:
#PIPELINE_MODE=full EXEC_MODE=dask python script.py

#Deposition only, all subjects, Dask:
#PIPELINE_MODE=deposition EXEC_MODE=dask python script.py

#PBPK only, all subjects, Dask (must set INPUT_DEPO):
#PIPELINE_MODE=pbpk EXEC_MODE=dask INPUT_DEPO=/path/to/deposition python script.py

#Slurm job array, one or many subjects per job:
#PIPELINE_MODE=full EXEC_MODE=slurm sbatch --array=1-40 script.py start_idx end_idx output_dir

#Local Pool, all subjects:
#PIPELINE_MODE=full EXEC_MODE=pool python script.py
#""""

In [ ]:
Make change to include a flag, task_by = 'Subject' or 'Product', to the run_pbbm_simulations function and split tasks accordingly
# some imports#

class Execution:
    def __init__(self, settings, products, variability):
        self.settings = settings
        self.products = products
        self.variability = variability

    def simulate_subjects(self, subject_ids, output_dir, step_mode='full', distributed=True, use_pool=False,
                         pool_cores=8, input_deposition_dir=None):
        subjects = [SubjectPhysiology(sid, self.variability, enable_variability=self.settings.ENABLE_VARIABILITY) for sid in subject_ids]
        os.makedirs(output_dir, exist_ok=True)

        if use_pool and len(subjects) > 1:
            def single_subj(sim_subject):
                self.simulate_subjects([sim_subject.id], output_dir, step_mode, distributed=False, use_pool=False)  # num_cores = 1 here
            with Pool(processes=pool_cores) as pool:
                pool.map(single_subj, subjects)
            return

        if step_mode == 'deposition':
            deposition_pool = run_deposition_simulations(
                subjects, self.products, num_cores=1 if not use_pool else pool_cores, distributed=distributed)
            for i, subject_id in enumerate(subject_ids):
                result = {
                    "subject_id": subject_id,
                    "deposition_pool": {subject_id: deposition_pool[subject_id]},
                }
                with open(os.path.join(output_dir, f'subject_{subject_id}_deposition.pkl'), "wb") as f:
                    pickle.dump(result, f)

        elif step_mode == 'pbpk':
            if input_deposition_dir is None:
                raise ValueError('PBPK-only mode requires input_deposition_dir!')
            for i, subject_id in enumerate(subject_ids):
                fn = os.path.join(input_deposition_dir, f'subject_{subject_id}_deposition.pkl')
                if not os.path.exists(fn):
                    print(f"Deposition result not found for subject {subject_id}: {fn}")
                    continue
                with open(fn, "rb") as f:
                    depo_data = pickle.load(f)
                deposition_pool = {subject_id: depo_data["deposition_pool"][subject_id]}
                simulation_outputs = run_pbbm_simulations(
                    [subjects[i]], self.products, deposition_pool, self.variability, self.settings,
                    num_cores=1 if not use_pool else pool_cores, distributed=distributed
                )
                # simulation_outputs is flat but contains variable chunk per subject!
                # For PBPK-only mode, typically only one subject at a time -> no chunking needed.
                result = {
                    "subject_id": subject_id,
                    "deposition_pool": deposition_pool,
                    "pbpk_outputs": simulation_outputs,
                }
                with open(os.path.join(output_dir, f'subject_{subject_id}_pbpk.pkl'), "wb") as f:
                    pickle.dump(result, f)
        else:  # full
            deposition_pool = run_deposition_simulations(
                subjects, self.products, num_cores=1 if not use_pool else pool_cores, distributed=distributed)
            simulation_outputs = run_pbbm_simulations(
                subjects, self.products, deposition_pool, self.variability, self.settings,
                num_cores=1 if not use_pool else pool_cores, distributed=distributed)

            # Build per subject chunk mapping where APIs per product can vary
            # Compute per subject, number of outputs for all (product, api) pairs
            idx = 0
            for subj_idx, subject_id in enumerate(subject_ids):
                n_api_this_subj = sum(len(product.apis_data) for product in self.products)
                pbpk_outputs_this_subject = simulation_outputs[idx: idx + n_api_this_subj]
                result = {
                    "subject_id": subject_id,
                    "deposition_pool": {subject_id: deposition_pool[subject_id]},
                    "pbpk_outputs": pbpk_outputs_this_subject,
                }
                with open(os.path.join(output_dir, f'subject_{subject_id}_sim.pkl'), "wb") as f:
                    pickle.dump(result, f)
                idx += n_api_this_subj  # increment start for next subject



    def subject_pipeline_dask(self, subject_id, output_dir, step_mode, input_deposition_dir):
        self.simulate_subjects([subject_id], output_dir, step_mode=step_mode, distributed=True, use_pool=False, input_deposition_dir=input_deposition_dir)

    def run_dask_parallel(self, N_SUBJECTS, output_dir, step_mode, n_jobs=25, input_deposition_dir=None):
        from dask_jobqueue import SLURMCluster
        from dask.distributed import Client, as_completed
        
        cluster = SLURMCluster(
            cores=4, memory='10GB', queue='your_queue', walltime='04:00:00', job_extra=['--exclusive']
        )
        cluster.scale(jobs=n_jobs)
        client = Client(cluster)
        futures = [
            client.submit(self.subject_pipeline_dask, sid, output_dir, step_mode, input_deposition_dir)
            for sid in range(1, N_SUBJECTS+1)
        ]
        for future in as_completed(futures):
            future.result()

    def run_slurm_array(self, start_idx, end_idx, output_dir, step_mode, input_deposition_dir=None):
        subject_ids = list(range(start_idx, end_idx+1))
        self.simulate_subjects(subject_ids, output_dir, step_mode=step_mode, distributed=True, use_pool=False, input_deposition_dir=input_deposition_dir)

    def run_local_pool(self, N_SUBJECTS=200, output_dir="", pool_cores=8, step_mode="full", input_deposition_dir=None):
        subject_ids = list(range(1, N_SUBJECTS+1))
        self.simulate_subjects(subject_ids, output_dir, step_mode=step_mode, distributed=False, use_pool=True, pool_cores=pool_cores, input_deposition_dir=input_deposition_dir)


# Run-pbbm and run-simulation functions, similar signature here

def run_pbbm_simulations(subjects, products, deposition_pool, variability, settings, num_cores = 1, distributed = False):
    pbpk_tasks = []
    for subject in subjects:
        for product in products:
            for api_data in product.apis_data:
                api_name = api_data['api'].name
                if subject.id in deposition_pool and product.name in deposition_pool[subject.id] and api_name in deposition_pool[subject.id][product.name]:
                    initial_dose = deposition_pool[subject.id][product.name][api_name]
                    pbpk_tasks.append((subject, product.name, api_data, initial_dose, variability, settings))

    start_time = time.time()
    if distributed:
        simulation_outputs_all = [run_single_pbbm_simulation(args) for args in pbpk_tasks]
    else:
        with Pool(processes=num_cores) as pool:
            simulation_outputs_all = pool.map(run_single_pbbm_simulation, pbpk_tasks)
    end_time = time.time()
    print(f"\nTotal PBPK simulation time: {end_time - start_time:.2f} seconds.")
  
    simulation_outputs = [o for o in simulation_outputs_all if o is not None]
    
    return simulation_outputs